# 基于Seq2Seq生成对联

In [96]:
import pandas as pd
import numpy as np
from tensorflow.python.layers.core import Dense
import tensorflow as tf

In [97]:
# 超参数
# Number of Epochs
epochs = 3000
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001

In [98]:
#处理数据
source = open('source.txt','w')
target = open('target.txt','w')

with open('对联.txt','r') as f:
    for line in f.readlines():
        up_down = line.strip().split(' ')
        source.write(up_down[0]+'\n')
        target.write(up_down[1]+'\n')      

In [99]:
with open('source.txt','r') as f:
    source_data = f.read()
with open('target.txt','r') as f:
    target_data = f.read()
    
#将上下联分开
source = source_data.split('\n')
target = target_data.split('\n')

/Users/caijie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.TextIOWrapper name='source.txt' mode='w' encoding='UTF-8'>
  import sys
/Users/caijie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: ResourceWarning: unclosed file <_io.TextIOWrapper name='target.txt' mode='w' encoding='UTF-8'>
  


In [100]:
#seq2seq模型中不能输入字符，输入的应该是数字 就是使用数字表示文本
#首先建立字符到数字和数字到字符的字典
def extract_character_vocab(data):
    '''
    参数:data
    返回:voc_int int_voc
    '''
    special_words = ['<PAD>','<UNK>','<GO>','<EOS>']
    #<PAD>用于字符补全，'<UNK>','<GO>'用于Decoder端序列中<UNK>代替一些未出现的词或者低频词
    set_words = list(set([character for line in data for character in line]))
    int_to_voc = {idx:value for idx,value in enumerate(special_words+set_words)}
    voc_to_int = {word:idx for idx,word in int_to_voc.items()}
    return voc_to_int,int_to_voc

source_letter_to_int,source_int_to_letter = extract_character_vocab(source+target)
target_letter_to_int,target_int_to_letter = extract_character_vocab(source+target)

In [101]:
#dict的get方法定义了两个参数（a,b） 如果dict中存在key-a则返回dict[a]否则返回b
#将字符用数字表示
source_int = [[source_letter_to_int.get(letter,source_letter_to_int['<UNK>']) for letter in line]for line in source]

target_int = [[target_letter_to_int.get(letter,target_letter_to_int['<UNK>']) for letter in line]for line in target]

# 按步骤建立Encoder和Decoder模型

In [102]:
#1、首先建立encoder层
def get_encoder_layer(input_data,rnn_size,num_layers,source_sequence_length,source_voc_size,encoding_embedding_size):
    
    #tf.contrib.layers.embed_sequence 将文章的的每一个字使用embedding表示
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data,source_voc_size,encoding_embedding_size)
    
    def get_LSTMCell(rnn_size):
        return tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
    
    cell = tf.contrib.rnn.MultiRNNCell([get_LSTMCell(rnn_size) for _ in range(num_layers)])
    
    encoder_output,encoder_state = tf.nn.dynamic_rnn(cell,encoder_embed_input,sequence_length=source_sequence_length,dtype = tf.float32)
    
    return encoder_output,encoder_state


In [103]:
#2、其次建立decoder层
#decoder层包含两个阶段 training和predict两个阶段共享参数
#需要在target句子前加一个<go>表示句子的开头，还需要将target中的最后一个字符去掉
def process_decoder_input(data,voc_to_int,batch_size):
    
    ending = tf.strided_slice(data,[0,0],[batch_size,-1],[1,1])#将target中的最后一个字符去掉
    #fill参数表示的（形状,填充的数字）
    decoder_input = tf.concat([tf.fill([batch_size,1],voc_to_int['<GO>']),ending],1)#target句子前加一个<go>表示句子的开头
    
    return decoder_input

In [112]:
def get_decoder_layer(target_letter_to_int,decoding_embedding_size,num_layers,rnn_size,
                   target_sequence_length,max_target_sequence_length,encoder_state,decoder_input):
    #1、embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size,decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings,decoder_input)
    #2、构造Decoder中的RNNCell单元
    def get_decode_cell(rnn_size):
        return tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
    
    cell = tf.contrib.rnn.MultiRNNCell([get_decode_cell(rnn_size) for _ in range(num_layers)])
    #3、Output全连接层
    output_layer = Dense(target_vocab_size,kernel_initializer=tf.truncated_normal_initializer(mean=0.1,stddev=0.1))
    #4、Training decoder
    with tf.variable_scope('decode'):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                            sequence_length = target_sequence_length,
                                                           time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,training_helper,encoder_state,output_layer)
        training_decoder_output,_,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,impute_finished=True,
                                                                        maximum_iterations=max_target_sequence_length)
    #5、Predicting decoder
    #与前一个decoder共享参数
    with tf.variable_scope('decode',reuse = True):
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']],dtype = tf.int32),[batch_size],name='start_token')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,start_tokens,
                                                                     target_letter_to_int['<EOS>'])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,predicting_helper,encoder_state,output_layer)
        predicting_decoder_output,_,_ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,impute_finished=True,
                                                                          maximum_iterations=max_target_sequence_length)
        return training_decoder_output,predicting_decoder_output

In [113]:
#上面已经把Encoder和Decoder建立完毕，之后使用seq2seq模型将其链接起来
def seq2seq_model(input_data,targets,lr,target_sequence_length,
                  max_target_sequence_length,source_sequence_length,
                  source_voc_size,target_vocab_size,encoder_embedding_size,decoder_embedding_size,rnn_size,num_layers):
    _,encoder_state = get_encoder_layer(input_data,
                                        rnn_size,
                                        num_layers,
                                        source_sequence_length,
                                        source_voc_size,
                                        encoding_embedding_size)
    decoder_input = process_decoder_input(targets,target_letter_to_int,batch_size)
    training_decoder_output,predicting_decoder_output = get_decoder_layer(target_letter_to_int,decoding_embedding_size,
                                                                          num_layers,rnn_size,
                   target_sequence_length,max_target_sequence_length,encoder_state,decoder_input)
    return training_decoder_output,predicting_decoder_output

In [114]:
#定义输入数据函数
def get_inputs():
    inputs = tf.placeholder(tf.int32,[None,None],name = 'inputs')
    targets = tf.placeholder(tf.int32,[None,None],name = 'targets')
    learning_rate = tf.placeholder(tf.float32,name = 'learning_rate')
    
    target_sequence_length = tf.placeholder(tf.int32,(None,),name = 'target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length,name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32,(None,),name = 'source_sequence_length')
    return inputs,targets,learning_rate,target_sequence_length,max_target_sequence_length,source_sequence_length

In [118]:
#定义loss和optimizer
train_graph = tf.Graph()

with train_graph.as_default():
    
    inputs,targets,lr,target_sequence_length,max_target_sequence_length,source_sequence_length = get_inputs()
    training_decoder_output,predicting_decoder_output = seq2seq_model(  inputs,
                                                                        targets,
                                                                        lr,
                                                                        target_sequence_length,
                                                                        max_target_sequence_length,
                                                                        source_sequence_length,
                                                                        len(source_letter_to_int),
                                                                        len(target_letter_to_int),
                                                                        encoding_embedding_size,
                                                                        decoding_embedding_size,
                                                                        rnn_size,
                                                                        num_layers)
    train_logits = tf.identity(training_decoder_output.rnn_output,'logits')
    predict_logits = tf.identity(predicting_decoder_output.sample_id,'predictions')
    masks = tf.sequence_mask(target_sequence_length,max_target_sequence_length,dtype = tf.float32,name='masks')
    with tf.name_scope('optimization'):
        cost = tf.contrib.seq2seq.sequence_loss(
        train_logits,
        targets,    
        masks)
        optimizer = tf.train.AdamOptimizer(lr)
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
    

/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullar

/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullar

In [119]:
def pad_sentence_batch(sentence_batch,pad_int):
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence+[pad_int] * (max_sentence-len(sentence))for sentence in sentence_batch]


In [122]:
def get_batches(targets,source,batch_size,source_pad_int,target_pad_int):
    for batch_i in range(0,len(source)//batch_size):
        start_i = batch_i * batch_size
        source_batch = sources[start_i:start_i + batch_size]
        target_batch = sources[start_i:start_i + batch_size]
        
        pad_source_batch = pad_sentence_batch(source_batch,source_pad_int)
        pad_target_batch = pad_sentence_batch(target_batch,source_pad_int)
        
        targets_length = []
        for target in target_batch:
            targets_length.append(len(target))
        source_length = []
        for source in source_length:
            source_length.append(len(source))
        yield pad_target_batch,pad_source_batch,targets_length,source_length

In [123]:
# Train
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]

# 留出一个batch进行验证
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]

(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target, valid_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']))

display_step = 50

checkpoint = "data/trained_model.ckpt"

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print()
    for epoch_i in range(1,epochs+1):
        for batch_i,(targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(get_batches(
            train_target,train_source,batch_size,source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']
        )):
            _,loss = sess.run([train_op,cost],feed_dict={
                input_data:sources_batch,
                targets:targets_batch,
                lr:learning_rate,
                target_sequence_length:targets_lengths,
                source_sequence_length:sources_lengths
            })

            if batch_i % display_step == 0:
                # 计算validation loss
                validation_loss = sess.run(
                    [cost],
                    {input_data: valid_sources_batch,
                     targets: valid_targets_batch,
                     lr: learning_rate,
                     target_sequence_length: valid_targets_lengths,
                     source_sequence_length: valid_sources_lengths})

                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs,
                              batch_i,
                              len(train_source) // batch_size,
                              loss,
                              validation_loss[0]))

    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')


# 预测
def source_to_seq(text):
    sequence_length = 7
    return [source_letter_to_int.get(word,source_letter_to_int['<UNK>']) for word in text] + [source_letter_to_int['<PAD>']] * (sequence_length - len(text))


input_word = '戌岁祝福万事顺'
text = source_to_seq(input_word)

checkpoint = "data/trained_model.ckpt"
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    loader = tf.train.import_meta_graph(checkpoint+'.meta')
    loader.restore(sess,checkpoint)

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')

    answer_logits = sess.run(logits, {input_data: [text] * batch_size,
                                      target_sequence_length: [len(input_word)] * batch_size,
                                      source_sequence_length: [len(input_word)] * batch_size})[0]

    pad = source_letter_to_int["<PAD>"]

    print('原始输入:', input_word)

    print('\nSource')
    print('  Word 编号:    {}'.format([i for i in text]))
    print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

    print('\nTarget')
    print('  Word 编号:       {}'.format([i for i in answer_logits if i != pad]))
    print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))


StopIteration: 

In [127]:
#鸡鸣万户晓 鹤舞一年春
#戌岁祝福万事顺 狗年兆丰五谷香
import tensorflow as tf
from tensorflow.python.layers.core import Dense

import numpy as np
import time


# 超参数
# Number of Epochs
epochs = 2000
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 3
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001

source = open("./source.txt",'w')
target = open("./target.txt",'w')

with open("./对联.txt",'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip().split(" ")
        source.write(line[0]+'\n')
        target.write(line[1]+'\n')
source.close()
target.close()

with open('./source.txt','r',encoding='utf-8') as f:
    source_data = f.read()

with open('./target.txt','r',encoding='utf-8') as f:
    target_data = f.read()

print(source_data.split('\n')[:10])
print(target_data.split('\n')[:10])


def extract_character_vocab(data):
    """
    :param data:
    :return: 字符映射表
    """
    special_words = ['<PAD>','<UNK>','<GO>','<EOS>']
    set_words = list(set([character for line in data.split('\n') for character in line]))
    int_to_vocab = {idx:word for idx,word in enumerate(special_words + set_words)}
    vocab_to_int = {word:idx for idx,word in int_to_vocab.items()}

    return int_to_vocab,vocab_to_int

# 得到输入和输出的字符映射表
source_int_to_letter,source_letter_to_int = extract_character_vocab(source_data+target_data)
target_int_to_letter,target_letter_to_int = extract_character_vocab(source_data+target_data)

# 将每一行转换成字符id的list
source_int = [[source_letter_to_int.get(letter,source_letter_to_int['<UNK>'])
               for letter in line] for line in source_data.split('\n')]

target_int = [[target_letter_to_int.get(letter, target_letter_to_int['<UNK>'])
               for letter in line] + [target_letter_to_int['<EOS>']] for line in target_data.split('\n')]

print(source_int)
print(target_int)

# 输入层
def get_inputs():

    inputs = tf.placeholder(tf.int32,[None,None],name='inputs')
    targets = tf.placeholder(tf.int32,[None,None],name='targets')
    learning_rate = tf.placeholder(tf.float32,name='learning_rate')

    # 定义target序列最大长度（之后target_sequence_length和source_sequence_length会作为feed_dict的参数）
    target_sequence_length = tf.placeholder(tf.int32,(None,),name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length,name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32,(None,),name='source_sequence_length')

    return inputs,targets,learning_rate,target_sequence_length,max_target_sequence_length,source_sequence_length


# Encoder
"""
在Encoder端，我们需要进行两步，第一步要对我们的输入进行Embedding，再把Embedding以后的向量传给RNN进行处理。

在Embedding中，我们使用tf.contrib.layers.embed_sequence，它会对每个batch执行embedding操作。
"""

def get_encoder_layer(input_data,rnn_size,num_layers,source_sequence_length,source_vocab_size,encoding_embedding_size):
    """
    构造Encoder层

    参数说明：
    - input_data: 输入tensor
    - rnn_size: rnn隐层结点数量
    - num_layers: 堆叠的rnn cell数量
    - source_sequence_length: 源数据的序列长度
    - source_vocab_size: 源数据的词典大小
    - encoding_embedding_size: embedding的大小
    """
    # https://www.tensorflow.org/versions/r1.4/api_docs/python/tf/contrib/layers/embed_sequence
    """
    embed_sequence(
    ids,
    vocab_size=None,
    embed_dim=None,
    unique=False,
    initializer=None,
    regularizer=None,
    trainable=True,
    scope=None,
    reuse=None
    )
    ids: [batch_size, doc_length] Tensor of type int32 or int64 with symbol ids.
    
    return : Tensor of [batch_size, doc_length, embed_dim] with embedded sequences.
    """
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data,source_vocab_size,encoding_embedding_size)

    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        return lstm_cell

    cell =  tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])

    encoder_output , encoder_state = tf.nn.dynamic_rnn(cell,encoder_embed_input,sequence_length=source_sequence_length,dtype=tf.float32)

    return encoder_output,encoder_state



def process_decoder_input(data,vocab_to_int,batch_size):

    ending = tf.strided_slice(data,[0,0],[batch_size,-1],[1,1])
    decoder_input = tf.concat([tf.fill([batch_size,1],vocab_to_int['<GO>']),ending],1)

    return decoder_input


def decoding_layer(target_letter_to_int,decoding_embedding_size,num_layers,rnn_size,
                   target_sequence_length,max_target_sequence_length,encoder_state,decoder_input):
    '''
    构造Decoder层

    参数：
    - target_letter_to_int: target数据的映射表
    - decoding_embedding_size: embed向量大小
    - num_layers: 堆叠的RNN单元数量
    - rnn_size: RNN单元的隐层结点数量
    - target_sequence_length: target数据序列长度
    - max_target_sequence_length: target数据序列最大长度
    - encoder_state: encoder端编码的状态向量
    - decoder_input: decoder端输入
    '''

    # 1. Embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size,decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings,decoder_input)

    # 构造Decoder中的RNN单元
    def get_decoder_cell(rnn_size):
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size,initializer=tf.random_uniform_initializer(-0.1,0.1,seed=2))
        return decoder_cell

    cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size) for _ in range(num_layers)])

    # Output全连接层
    # target_vocab_size定义了输出层的大小
    output_layer = Dense(target_vocab_size,kernel_initializer=tf.truncated_normal_initializer(mean=0.1,stddev=0.1))


    # 4. Training decoder
    with tf.variable_scope("decode"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs = decoder_embed_input,
                                                            sequence_length = target_sequence_length,
                                                            time_major = False)


        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,training_helper,encoder_state,output_layer)
        training_decoder_output,_,_ = tf.contrib.seq2seq.dynamic_decode(training_decoder,impute_finished=True,
                                                                        maximum_iterations = max_target_sequence_length)


    # 5. Predicting decoder
    # 与training共享参数

    with tf.variable_scope("decode",reuse=True):
        # 创建一个常量tensor并复制为batch_size的大小
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']],dtype=tf.int32),[batch_size],name='start_token')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,start_tokens,target_letter_to_int['<EOS>'])

        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                             predicting_helper,
                                                             encoder_state,
                                                             output_layer)
        predicting_decoder_output,_,_ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder,impute_finished = True,
                                                                          maximum_iterations = max_target_sequence_length)


    return training_decoder_output,predicting_decoder_output

# 上面已经构建完成Encoder和Decoder，下面将这两部分连接起来，构建seq2seq模型
def seq2seq_model(input_data,targets,lr,target_sequence_length,max_target_sequence_length,
                  source_sequence_length,source_vocab_size,target_vocab_size,encoder_embedding_size,
                  decoder_embedding_size,rnn_size,num_layers):

    _,encoder_state = get_encoder_layer(input_data,
                                        rnn_size,
                                        num_layers,
                                        source_sequence_length,
                                        source_vocab_size,
                                        encoding_embedding_size)

    decoder_input = process_decoder_input(targets,target_letter_to_int,batch_size)

    training_decoder_output,predicting_decoder_output = decoding_layer(target_letter_to_int,
                                                                       decoding_embedding_size,
                                                                       num_layers,
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       encoder_state,
                                                                       decoder_input)

    return training_decoder_output,predicting_decoder_output






# 构造graph
train_graph = tf.Graph()

with train_graph.as_default():
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_inputs()

    training_decoder_output, predicting_decoder_output = seq2seq_model(input_data,
                                                                       targets,
                                                                       lr,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       source_sequence_length,
                                                                       len(source_letter_to_int),
                                                                       len(target_letter_to_int),
                                                                       encoding_embedding_size,
                                                                       decoding_embedding_size,
                                                                       rnn_size,
                                                                       num_layers)

    training_logits = tf.identity(training_decoder_output.rnn_output,'logits')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id,name='predictions')

    #mask是权重的意思
    #tf.sequence_mask([1, 3, 2], 5)  # [[True, False, False, False, False],
                                #  [True, True, True, False, False],
                                #  [True, True, False, False, False]]
    masks = tf.sequence_mask(target_sequence_length,max_target_sequence_length,dtype=tf.float32,name="masks")

    # logits: A Tensor of shape [batch_size, sequence_length, num_decoder_symbols] and dtype float.
    # The logits correspond to the prediction across all classes at each timestep.
    #targets: A Tensor of shape [batch_size, sequence_length] and dtype int.
    # The target represents the true class at each timestep.
    #weights: A Tensor of shape [batch_size, sequence_length] and dtype float.
    # weights constitutes the weighting of each prediction in the sequence. When using weights as masking,
    # set all valid timesteps to 1 and all padded timesteps to 0, e.g. a mask returned by tf.sequence_mask.
    with tf.name_scope("optimization"):
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks
        )

        optimizer = tf.train.AdamOptimizer(lr)

        # minimize函数用于添加操作节点，用于最小化loss，并更新var_list.
        # 该函数是简单的合并了compute_gradients()与apply_gradients()函数返回为一个优化更新后的var_list，
        # 如果global_step非None，该操作还会为global_step做自增操作

        #这里将minimize拆解为了以下两个部分：

        # 对var_list中的变量计算loss的梯度 该函数为函数minimize()的第一部分，返回一个以元组(gradient, variable)组成的列表
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        # 将计算出的梯度应用到变量上，是函数minimize()的第二部分，返回一个应用指定的梯度的操作Operation，对global_step做自增操作
        train_op = optimizer.apply_gradients(capped_gradients)


def pad_sentence_batch(sentence_batch,pad_int):
    '''
    对batch中的序列进行补全，保证batch中的每行都有相同的sequence_length

    参数：
    - sentence batch
    - pad_int: <PAD>对应索引号
    '''
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(targets,sources,batch_size,source_pad_int,target_pad_int):

    for batch_i in range(0,len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i : start_i + batch_size]
        targets_batch = targets[start_i : start_i + batch_size]

        pad_sources_batch = np.array(pad_sentence_batch(sources_batch,source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch,target_pad_int))

        targets_lengths = []
        for target in targets_batch:
            targets_lengths.append(len(target))

        source_lengths = []
        for source in sources_batch:
            source_lengths.append(len(source))

        yield pad_targets_batch,pad_sources_batch,targets_lengths,source_lengths



# Train
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]

# 留出一个batch进行验证
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]

(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target, valid_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']))

display_step = 50

checkpoint = "data/trained_model.ckpt"

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print()
    for epoch_i in range(1,epochs+1):
        for batch_i,(targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(get_batches(
            train_target,train_source,batch_size,source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']
        )):
            _,loss = sess.run([train_op,cost],feed_dict={
                input_data:sources_batch,
                targets:targets_batch,
                lr:learning_rate,
                target_sequence_length:targets_lengths,
                source_sequence_length:sources_lengths
            })

            if batch_i % display_step == 0:
                # 计算validation loss
                validation_loss = sess.run(
                    [cost],
                    {input_data: valid_sources_batch,
                     targets: valid_targets_batch,
                     lr: learning_rate,
                     target_sequence_length: valid_targets_lengths,
                     source_sequence_length: valid_sources_lengths})

                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs,
                              batch_i,
                              len(train_source) // batch_size,
                              loss,
                              validation_loss[0]))

    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')


# 预测
def source_to_seq(text):
    sequence_length = 7
    return [source_letter_to_int.get(word,source_letter_to_int['<UNK>']) for word in text] + [source_letter_to_int['<PAD>']] * (sequence_length - len(text))


input_word = '戌岁祝福万事顺'
text = source_to_seq(input_word)

checkpoint = "data/trained_model.ckpt"
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    loader = tf.train.import_meta_graph(checkpoint+'.meta')
    loader.restore(sess,checkpoint)

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')

    answer_logits = sess.run(logits, {input_data: [text] * batch_size,
                                      target_sequence_length: [len(input_word)] * batch_size,
                                      source_sequence_length: [len(input_word)] * batch_size})[0]

    pad = source_letter_to_int["<PAD>"]

    print('原始输入:', input_word)

    print('\nSource')
    print('  Word 编号:    {}'.format([i for i in text]))
    print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

    print('\nTarget')
    print('  Word 编号:       {}'.format([i for i in answer_logits if i != pad]))
    print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))


['承上下求索志', '除旧岁破旧俗', '处处春光济美', '处处欢歌遍地', '处处明山秀水', '窗外红梅最艳', '创造万千气象', '春到碧桃树上', '为江山添秀色', '爆竹一声除旧']
['绘春秋振兴图', '迎新年树新风', '年年人物风流', '家家喜笑连天', '家家笑语欢歌', '心头美景尤佳', '建设两个文明', '莺歌绿柳楼前', '与日月争光辉', '桃符万户更新']
[[279, 206, 521, 263, 580, 22], [21, 408, 43, 161, 408, 462], [200, 200, 653, 455, 223, 597], [200, 200, 542, 726, 118, 398], [200, 200, 356, 490, 243, 723], [260, 51, 492, 300, 216, 187], [291, 464, 525, 33, 447, 673], [653, 561, 96, 459, 126, 206], [205, 475, 490, 25, 243, 306], [289, 456, 647, 457, 21, 408], [352, 435, 374, 158, 436, 644], [352, 435, 33, 695, 327, 436], [660, 224, 503, 653, 552, 600], [435, 176, 455, 639, 43, 505], [169, 159, 707, 452, 732, 176], [480, 467, 282, 314, 626, 27], [583, 705, 44, 66, 33, 43], [583, 69, 492, 300, 375, 625], [569, 669, 299, 337, 595, 187], [569, 669, 299, 337, 595, 187], [569, 669, 358, 653, 120, 187], [569, 123, 635, 653, 120, 187], [569, 478, 44, 726, 352, 435], [653, 561, 670, 446, 547, 106], [653, 314, 215, 118, 608, 558], [653, 314, 653

/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullar

/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullar


Epoch   1/2000 Batch    0/1 - Training Loss:  6.599  - Validation loss:  6.597
Epoch   2/2000 Batch    0/1 - Training Loss:  6.595  - Validation loss:  6.594
Epoch   3/2000 Batch    0/1 - Training Loss:  6.592  - Validation loss:  6.591
Epoch   4/2000 Batch    0/1 - Training Loss:  6.588  - Validation loss:  6.588
Epoch   5/2000 Batch    0/1 - Training Loss:  6.583  - Validation loss:  6.584
Epoch   6/2000 Batch    0/1 - Training Loss:  6.578  - Validation loss:  6.579
Epoch   7/2000 Batch    0/1 - Training Loss:  6.571  - Validation loss:  6.573
Epoch   8/2000 Batch    0/1 - Training Loss:  6.563  - Validation loss:  6.565
Epoch   9/2000 Batch    0/1 - Training Loss:  6.552  - Validation loss:  6.555
Epoch  10/2000 Batch    0/1 - Training Loss:  6.538  - Validation loss:  6.543
Epoch  11/2000 Batch    0/1 - Training Loss:  6.521  - Validation loss:  6.526
Epoch  12/2000 Batch    0/1 - Training Loss:  6.498  - Validation loss:  6.506
Epoch  13/2000 Batch    0/1 - Training Loss:  6.469

Epoch 105/2000 Batch    0/1 - Training Loss:  4.591  - Validation loss:  6.555
Epoch 106/2000 Batch    0/1 - Training Loss:  4.587  - Validation loss:  6.559
Epoch 107/2000 Batch    0/1 - Training Loss:  4.582  - Validation loss:  6.564
Epoch 108/2000 Batch    0/1 - Training Loss:  4.578  - Validation loss:  6.569
Epoch 109/2000 Batch    0/1 - Training Loss:  4.573  - Validation loss:  6.573
Epoch 110/2000 Batch    0/1 - Training Loss:  4.569  - Validation loss:  6.578
Epoch 111/2000 Batch    0/1 - Training Loss:  4.564  - Validation loss:  6.583
Epoch 112/2000 Batch    0/1 - Training Loss:  4.560  - Validation loss:  6.588
Epoch 113/2000 Batch    0/1 - Training Loss:  4.556  - Validation loss:  6.594
Epoch 114/2000 Batch    0/1 - Training Loss:  4.551  - Validation loss:  6.598
Epoch 115/2000 Batch    0/1 - Training Loss:  4.547  - Validation loss:  6.603
Epoch 116/2000 Batch    0/1 - Training Loss:  4.543  - Validation loss:  6.608
Epoch 117/2000 Batch    0/1 - Training Loss:  4.538 

...

Epoch 1773/2000 Batch    0/1 - Training Loss:  0.109  - Validation loss: 12.582
Epoch 1774/2000 Batch    0/1 - Training Loss:  0.109  - Validation loss: 12.585
Epoch 1775/2000 Batch    0/1 - Training Loss:  0.109  - Validation loss: 12.587
Epoch 1776/2000 Batch    0/1 - Training Loss:  0.108  - Validation loss: 12.592
Epoch 1777/2000 Batch    0/1 - Training Loss:  0.108  - Validation loss: 12.596
Epoch 1778/2000 Batch    0/1 - Training Loss:  0.108  - Validation loss: 12.596
Epoch 1779/2000 Batch    0/1 - Training Loss:  0.108  - Validation loss: 12.599
Epoch 1780/2000 Batch    0/1 - Training Loss:  0.108  - Validation loss: 12.599
Epoch 1781/2000 Batch    0/1 - Training Loss:  0.107  - Validation loss: 12.601
Epoch 1782/2000 Batch    0/1 - Training Loss:  0.107  - Validation loss: 12.604
Epoch 1783/2000 Batch    0/1 - Training Loss:  0.107  - Validation loss: 12.604
Epoch 1784/2000 Batch    0/1 - Training Loss:  0.107  - Validation loss: 12.609
Epoch 1785/2000 Batch    0/1 - Training 

Epoch 1877/2000 Batch    0/1 - Training Loss:  0.089  - Validation loss: 12.776
Epoch 1878/2000 Batch    0/1 - Training Loss:  0.089  - Validation loss: 12.778
Epoch 1879/2000 Batch    0/1 - Training Loss:  0.089  - Validation loss: 12.779
Epoch 1880/2000 Batch    0/1 - Training Loss:  0.089  - Validation loss: 12.784
Epoch 1881/2000 Batch    0/1 - Training Loss:  0.089  - Validation loss: 12.782
Epoch 1882/2000 Batch    0/1 - Training Loss:  0.088  - Validation loss: 12.785
Epoch 1883/2000 Batch    0/1 - Training Loss:  0.088  - Validation loss: 12.787
Epoch 1884/2000 Batch    0/1 - Training Loss:  0.088  - Validation loss: 12.789
Epoch 1885/2000 Batch    0/1 - Training Loss:  0.088  - Validation loss: 12.793
Epoch 1886/2000 Batch    0/1 - Training Loss:  0.088  - Validation loss: 12.791
Epoch 1887/2000 Batch    0/1 - Training Loss:  0.088  - Validation loss: 12.793
Epoch 1888/2000 Batch    0/1 - Training Loss:  0.087  - Validation loss: 12.796
Epoch 1889/2000 Batch    0/1 - Training 

Epoch 1981/2000 Batch    0/1 - Training Loss:  0.074  - Validation loss: 12.946
Epoch 1982/2000 Batch    0/1 - Training Loss:  0.074  - Validation loss: 12.948
Epoch 1983/2000 Batch    0/1 - Training Loss:  0.074  - Validation loss: 12.945
Epoch 1984/2000 Batch    0/1 - Training Loss:  0.074  - Validation loss: 12.949
Epoch 1985/2000 Batch    0/1 - Training Loss:  0.073  - Validation loss: 12.956
Epoch 1986/2000 Batch    0/1 - Training Loss:  0.073  - Validation loss: 12.954
Epoch 1987/2000 Batch    0/1 - Training Loss:  0.073  - Validation loss: 12.951
Epoch 1988/2000 Batch    0/1 - Training Loss:  0.073  - Validation loss: 12.954
Epoch 1989/2000 Batch    0/1 - Training Loss:  0.073  - Validation loss: 12.959
Epoch 1990/2000 Batch    0/1 - Training Loss:  0.073  - Validation loss: 12.961
Epoch 1991/2000 Batch    0/1 - Training Loss:  0.072  - Validation loss: 12.961
Epoch 1992/2000 Batch    0/1 - Training Loss:  0.072  - Validation loss: 12.964
Epoch 1993/2000 Batch    0/1 - Training 

/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


INFO:tensorflow:Restoring parameters from data/trained_model.ckpt
原始输入: 戌岁祝福万事顺

Source
  Word 编号:    [517, 43, 112, 111, 525, 699, 622]
  Input Words: 戌 岁 祝 福 万 事 顺

Target
  Word 编号:       [522, 43, 630, 111, 666, 441, 62]
  Response Words: 狗 岁 年 福 足 食 人
